In [1]:
from transformers import file_utils
import transformers
import torch

In [2]:
from huggingface_hub import login
login(token = '')

In [3]:
print(transformers.__version__)

4.45.2


In [4]:
model_id = 'meta-llama/Meta-Llama-3-8B-instruct'

# Create a text generation pipeline that uses the model defined above
pipeline = transformers.pipeline(
    'text-generation',
    model = model_id,
    model_kwargs = {'torch_dtype': torch.bfloat16},
    device_map = 'auto',
    pad_token_id = 50256
)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

C:\Users\oliva\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\oliva\.cache\huggingface\hub\models--meta-llama--Meta-Llama-3-8B-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [6]:
messages = [
    # Telling the model how to behave and how to respond to the user when given information.
    # Describing it's main purpose without much detail on how it got it's conclusion.
    {"role": "system", "content": " you are a counterfacual estimator of the individual that user gives. \
    The user provides the features of the real-world individual. Then you estimate the counterpart based on\
    the requested feature like sex, race, age, and etc. do not provide any explanation on how you estimate, \
    just give the features of the counterpart same as the provided features by the user." },

    # User requesting counterfactual estimates.
    {"role": "user", "content":"I am Male, 42, income 80K. what would be my 41 years old counterfactual?"},
    # Add other census features to user prompt to  create counterfactuals.
]

In [ ]:
# Create a similar message to the one above but replace protected_params numbers to their feature name.

# the name of each feature
feature_name = ["age", "workclass", "fnlwgt", "education", "marital_status", "occupation", "relationship", "race", "sex", "capital_gain",
                                                                      "capital_loss", "hours_per_week", "native_country"]

messages = []

In [ ]:
# Converts the conversation in messages into a single text prompt to give to the llm
prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# Generates the prompts response.
outputs = pipeline(
    prompt,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.1,
    top_p=0.99,
)
print(outputs[0]["generated_text"][len(prompt):])